In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB


%matplotlib inline

In [26]:
## Convert to NumpyArray for SVM features
#def toNumpyArray(sequence_bagged):
#    length = max(map(len, sequence_bagged))
 #   return numpy.array([words+['space']*(length-len(words)) for words in sequence_bagged])

In [27]:
trainmain_df = pd.read_csv('./data2/train_data_bagged.csv')
testmain_df = pd.read_csv('./data2/test_data_bagged.csv')
trainmain_df.rename({'0':'sequence','label':'label'},axis=1,inplace=True)
testmain_df.rename({'0':'sequence'},axis=1,inplace=True)
testmain_df.columns

Index(['sequence'], dtype='object')

In [28]:
testmain_df

,sequence
0,AATC ATCT TCTC CTCT TCTG CTGT TGTT GTTG TTGT T...
1,TTCT TCTC CTCT TCTA CTAA TAAG AAGA AGAA GAAA A...
2,ACAT CATT ATTT TTTT TTTA TTAC TACG ACGG CGGT G...
3,CACT ACTG CTGA TGAA GAAA AAAA AAAA AAAA AAAA A...
4,GGGA GGAG GAGT AGTG GTGG TGGC GGCA GCAA CAAC A...
...,...
173,TAAA AAAT AATT ATTG TTGC TGCA GCAT CATG ATGT T...
174,GGAT GATG ATGT TGTG GTGC TGCT GCTT CTTA TTAG T...
175,ATTC TTCA TCAG CAGA AGAA GAAA AAAT AATT ATTA T...
176,ACGG CGGT GGTT GTTT TTTT TTTA TTAA TAAG AAGT A...


In [30]:
trainmain_df['sequence'].values

array(['TTAA TAAT AATT ATTT TTTG TTGT TGTC GTCC TCCT CCTT CTTA TTAT TATT ATTT TTTG TTGA TGAT GATT ATTA TTAA TAAG AAGA AGAA GAAG AAGA AGAA GAAT AATA ATAA TAAA AAAT AATC ATCT TCTT CTTA TTAT TATA ATAT TATA ATAT TATA ATAG TAGA AGAT GATT ATTT TTTA TTAC TACA ACAA CAAT AATC ATCT TCTA CTAT TATC ATCG TCGC CGCC GCCT CCTA CTAA TAAA AAAC AACT ACTT CTTC TTCA TCAG CAGC AGCC GCCA CCAC CACT ACTT CTTA TTAA TAAT AATC ATCA TCAA CAAT AATA ATAA TAAT AATC ATCG TCGC CGCG GCGA CGAC GACA ACAA CAAT AATG ATGA TGAT GATT ATTA TTAT TATT ATTT TTTT TTTC TTCT TCTA CTAC TACA ACAA CAAA AAAT AATC ATCA TCAT CATA ATAA TAAA AAAG AAGA AGAT GATA ATAT TATT ATTG TTGG TGGA GGAA GAAC AACT ACTT CTTT TTTA TTAT TATA ATAT TATT ATTT TTTT TTTA TTAT TATT ATTT TTTT TTTT TTTG TTGG TGGA GGAG GAGC AGCT GCTT CTTG TTGA TGAG GAGC AGCT GCTG CTGG TGGA GGAA GAAT AATA ATAG TAGT AGTT GTTG TTGG TGGA GGAA GAAC AACA ACAT CATC ATCT TCTT CTTT TTTA TTAA TAAG AAGA AGAA GAAT AATT ATTT TTTT TTTA TTAA TAAT AATT ATTC TTCG TCGA CGAG GAGC AGCT GCTG CTGA TGAA GA

In [31]:
X = trainmain_df['sequence'].values
y = trainmain_df['label'].values

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42)

In [33]:
X_train[0]

'ATGC TGCA GCAA CAAC AACC ACCA CCAG CAGC AGCC GCCA CCAA CAAT AATA ATAC TACA ACAC CACT ACTT CTTT TTTT TTTA TTAG TAGA AGAG GAGG AGGC GGCT GCTG CTGG TGGT GGTA GTAC TACT ACTG CTGA TGAT GATC ATCG TCGA CGAA GAAA AAAA AAAC AACT ACTC CTCG TCGA CGAA GAAA AAAC AACC ACCA CCAC CACT ACTT CTTG TTGA TGAC GACA ACAC CACT ACTC CTCG TCGA CGAA GAAG AAGT AGTA GTAA TAAG AAGT AGTT GTTA TTAA TAAT AATA ATAT TATT ATTC TTCC TCCT CCTC CTCT TCTA CTAA TAAA AAAT AATA ATAG TAGG AGGT GGTG GTGA TGAA GAAA AAAT AATA ATAC TACA ACAA CAAA AAAT AATT ATTC TTCG TCGT CGTA GTAG TAGC AGCA GCAA CAAA AAAC AACT ACTG CTGC TGCG GCGG CGGC GGCC GCCT CCTT CTTA TTAA TAAA AAAT AATC ATCG TCGA CGAA GAAT AATA ATAG TAGC AGCC GCCG CCGG CGGA GGAA GAAG AAGT AGTG GTGG TGGA GGAA GAAC AACA ACAT CATG ATGA TGAA GAAC AACT ACTT CTTC TTCA TCAA CAAT AATG ATGT TGTT GTTA TTAT TATG ATGA TGAA GAAC AACG ACGT CGTC GTCG TCGC CGCT GCTT CTTA TTAC TACC ACCA CCAC CACC ACCT CCTG CTGC TGCA GCAA CAAG AAGC AGCC GCCC CCCT CCTA CTAA TAAA AAAA AAAT AATA ATAT TATC ATCG TCGA

In [34]:
tfid = TfidfVectorizer(ngram_range=(1,2),min_df=0.2,max_df=0.8,max_features=128)

In [35]:
tfid.fit(X_train)

TfidfVectorizer(max_df=0.8, max_features=128, min_df=0.2, ngram_range=(1, 2))

In [36]:
X_train_tok = tfid.transform(X_train)
X_train_df = pd.DataFrame(X_train_tok.toarray(),
                          columns=tfid.get_feature_names())
X_train_df

,aaaa aaaa,aaaa aaat,aaat aata,aaat aatt,aacg,aact,aaga,aagc,aagt,aata ataa,...,ttca,ttcc,ttcg,ttct,ttga,ttgc,ttgg,ttta ttaa,tttt ttta,tttt tttt
0,0.053106,0.096682,0.206777,0.051899,0.052254,0.228446,0.000000,0.047635,0.176317,0.000000,...,0.133336,0.101263,0.048162,0.095130,0.137578,0.046106,0.047416,0.000000,0.055309,0.000000
1,0.045995,0.125605,0.089545,0.044950,0.045258,0.118715,0.041009,0.000000,0.038177,0.147028,...,0.000000,0.087705,0.000000,0.041196,0.079438,0.039933,0.082134,0.000000,0.095807,0.571906
2,0.000000,0.112617,0.120428,0.120904,0.000000,0.035480,0.073537,0.000000,0.068459,0.000000,...,0.069028,0.039318,0.037400,0.036936,0.071224,0.000000,0.073641,0.000000,0.000000,0.170922
3,0.000000,0.000000,0.000000,0.060174,0.181757,0.052974,0.054898,0.055230,0.102215,0.000000,...,0.051532,0.058705,0.279205,0.055149,0.000000,0.213830,0.054976,0.128306,0.000000,0.000000
4,0.381475,0.173625,0.092834,0.186402,0.000000,0.164100,0.042515,0.128317,0.079159,0.050809,...,0.079817,0.000000,0.000000,0.085418,0.041178,0.082799,0.042575,0.149048,0.049663,0.098819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16945,0.000000,0.000000,0.112197,0.000000,0.000000,0.049582,0.051383,0.051694,0.047835,0.000000,...,0.144696,0.000000,0.052265,0.051617,0.199066,0.000000,0.000000,0.060045,0.060021,0.298574
16946,0.000000,0.000000,0.047605,0.000000,0.096240,0.126224,0.174411,0.000000,0.040592,0.052109,...,0.122788,0.046626,0.044352,0.043802,0.042231,0.000000,0.130994,0.254768,0.152801,0.253367
16947,0.049916,0.090875,0.000000,0.146345,0.000000,0.042945,0.000000,0.000000,0.082864,0.053187,...,0.167104,0.000000,0.135808,0.044708,0.043105,0.000000,0.044568,0.000000,0.103975,0.103443
16948,0.056108,0.051074,0.000000,0.000000,0.000000,0.144816,0.000000,0.050328,0.093142,0.000000,...,0.000000,0.160481,0.000000,0.050254,0.048452,0.146137,0.100192,0.000000,0.000000,0.000000


In [37]:
X_test_tok = tfid.transform(X_test)
X_test_df = pd.DataFrame(X_test_tok.toarray(),
                          columns=tfid.get_feature_names())
X_test_df

,aaaa aaaa,aaaa aaat,aaat aata,aaat aatt,aacg,aact,aaga,aagc,aagt,aata ataa,...,ttca,ttcc,ttcg,ttct,ttga,ttgc,ttgg,ttta ttaa,tttt ttta,tttt tttt
0,0.159492,0.096788,0.207004,0.103911,0.052312,0.091479,0.000000,0.047688,0.132383,0.056648,...,0.088989,0.050687,0.000000,0.095234,0.045910,0.046157,0.047468,0.000000,0.055370,0.055087
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033738,0.135768,0.031408,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.032677,0.229967,0.067571,0.000000,0.000000,0.000000
2,0.240543,0.131377,0.280979,0.094030,0.094674,0.082780,0.042893,0.000000,0.039931,0.307568,...,0.000000,0.045867,0.043630,0.129267,0.083088,0.083535,0.085908,0.050124,0.100209,0.049849
3,0.054628,0.000000,0.159529,0.000000,0.161258,0.000000,0.048706,0.098002,0.226716,0.116417,...,0.045720,0.104167,0.049543,0.000000,0.000000,0.047428,0.048775,0.113835,0.113790,0.056605
4,0.119634,0.000000,0.000000,0.058457,0.176573,0.051463,0.106665,0.107310,0.049650,0.000000,...,0.050062,0.057030,0.108496,0.107152,0.051655,0.103865,0.000000,0.186970,0.062299,0.123961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5645,0.054929,0.050001,0.053469,0.107360,0.216192,0.094515,0.146922,0.147812,0.091185,0.058528,...,0.137914,0.000000,0.049815,0.000000,0.094867,0.047689,0.147130,0.057230,0.000000,0.113831
5646,0.052424,0.000000,0.051030,0.051232,0.154749,0.135307,0.046740,0.000000,0.000000,0.111718,...,0.043875,0.000000,0.047543,0.187816,0.135811,0.045514,0.140420,0.000000,0.109197,0.108640
5647,0.149716,0.136283,0.145736,0.048771,0.000000,0.085871,0.222476,0.044764,0.041423,0.106351,...,0.000000,0.000000,0.090519,0.044698,0.129287,0.043327,0.000000,0.051996,0.051976,0.051710
5648,0.000000,0.103243,0.110404,0.000000,0.055800,0.000000,0.202246,0.050868,0.047070,0.000000,...,0.000000,0.000000,0.102860,0.050792,0.048971,0.147704,0.050633,0.118171,0.118124,0.176282


In [39]:
testmain_df_tok = tfid.transform(testmain_df['sequence'].values)
testmain_df2 = pd.DataFrame(testmain_df_tok.toarray(),
                          columns=tfid.get_feature_names())
testmain_df2

,aaaa aaaa,aaaa aaat,aaat aata,aaat aatt,aacg,aact,aaga,aagc,aagt,aata ataa,...,ttca,ttcc,ttcg,ttct,ttga,ttgc,ttgg,ttta ttaa,tttt ttta,tttt tttt
0,0.138221,0.041940,0.134547,0.135079,0.045335,0.000000,0.000000,0.000000,0.076485,0.098186,...,0.115680,0.043927,0.000000,0.123799,0.039787,0.040001,0.000000,0.144013,0.143956,0.000000
1,0.000000,0.051899,0.000000,0.111437,0.056100,0.049052,0.101667,0.357989,0.141970,0.060751,...,0.000000,0.054358,0.103413,0.153197,0.000000,0.049500,0.000000,0.000000,0.059380,0.118153
2,0.059815,0.054449,0.116451,0.175368,0.000000,0.257310,0.053331,0.160962,0.099297,0.063736,...,0.100122,0.057029,0.162741,0.053575,0.000000,0.207726,0.000000,0.062322,0.124595,0.000000
3,0.410621,0.037378,0.079941,0.200644,0.000000,0.141311,0.109832,0.110497,0.068166,0.218766,...,0.000000,0.039149,0.000000,0.073556,0.070918,0.000000,0.036663,0.042783,0.000000,0.170190
4,0.113201,0.051522,0.055096,0.000000,0.222771,0.000000,0.100929,0.101540,0.093960,0.000000,...,0.142111,0.161890,0.102662,0.152085,0.048877,0.000000,0.050536,0.058972,0.058949,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0.000000,0.100412,0.000000,0.053901,0.271349,0.047452,0.000000,0.148418,0.000000,0.058769,...,0.184640,0.000000,0.000000,0.049400,0.000000,0.095769,0.049245,0.000000,0.000000,0.000000
174,0.102205,0.093035,0.099489,0.049941,0.050283,0.087932,0.136688,0.045839,0.084833,0.108903,...,0.128307,0.146166,0.000000,0.000000,0.088259,0.088734,0.045627,0.000000,0.000000,0.000000
175,0.192061,0.218537,0.093478,0.187695,0.094491,0.082620,0.000000,0.000000,0.079708,0.051162,...,0.080370,0.045778,0.000000,0.086011,0.082927,0.125060,0.042871,0.050027,0.050008,0.149256
176,0.000000,0.000000,0.048135,0.096650,0.048656,0.085087,0.044088,0.000000,0.041044,0.158069,...,0.124155,0.000000,0.089691,0.044290,0.085403,0.042931,0.044151,0.103042,0.154502,0.256188


In [41]:
svc = SVC()
svc.fit(X_train_tok,y_train)
cross_val_score(svc,X_train_tok,y_train,cv=5).mean()

0.7732743362831859

In [42]:
y_pred = svc.predict(X_test_tok)

In [43]:
accuracy_score(y_test, y_pred)

0.7690265486725664

In [44]:
y_pred_test = svc.predict(testmain_df_tok)

In [45]:
results_SVM = pd.DataFrame(y_pred_test)

In [48]:
original_SVM = pd.read_csv('./data2/test_data_bagged.csv')

In [49]:
original_SVM['predictions'] = y_pred_test

In [50]:
original_SVM.to_csv('./data/test2.csv', index=False)

In [52]:
original_SVM['predictions'].value_counts()

1    93
0    85
Name: predictions, dtype: int64